In [2]:
import pandas as pd

df = pd.read_excel('../data/cities_streetview_run_1.xlsx', engine='openpyxl')

# Filter the rows based on the date condition
print("CITIES STREETVIEWED: ")
print("within the last year: ",len(df[df['date'] >= '2022-11']))
print("within the last 6 months: ",len(df[df['date'] >= '2023-05']))
print("within the past 3 months: ",len(df[df['date'] >= '2023-08']))
print("within the past month: ",len(df[df['date'] >= '2023-10']))

CITIES STREETVIEWED: 
within the last year:  273
within the last 6 months:  137
within the past 3 months:  45
within the past month:  3


In [ ]:
from dotenv import load_dotenv
import os
import requests
import random
from utils import request_streetview

load_dotenv()

GOOGLE_MAPS_API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")

## GRAB SINGLE IMAGE, AND PASS IT TO THE MODEL
lat = 48.486132
lng = -123.325421

parameters = {
    "location": f'{lat},{lng}',
    "size": '640x300',
    "return_error_code": 'true',
    "key": GOOGLE_MAPS_API_KEY,
    "fov": 120,
    "heading": 0,
    "pitch": 0,
    "source": "outdoor",
    "radius": 50
}
# pano_id are dynamic, use location instead

id = random.randint(0, 1000)
file_base_path = f"../images/sample_image{id}"

parameters['heading'] = 0
request_streetview(parameters, f"{file_base_path}_N.jpg")

parameters['heading'] = 90
request_streetview(parameters, f"{file_base_path}_E.jpg")

parameters['heading'] = 180
request_streetview(parameters, f"{file_base_path}_S.jpg")

parameters['heading'] = 270
request_streetview(parameters, f"{file_base_path}_W.jpg")


In [ ]:
import pandas as pd

load_dotenv()
GOOGLE_MAPS_API_KEY = os.getenv('GOOGLE_MAPS_API_KEY')
URL_SIGNING_SECRET = os.getenv('URL_SIGNING_SECRET')

full_df = pd.read_excel('north-america-cities.csv')
filtered_df = pd.read_excel('cities_streetview_full_run.xlsx', engine='openpyxl')

# Takes city and location, and returns list of unique coordinates where streetview images were taken
def streetview_gridsearch(city, lat, lng):
    DEGREE_IN_METERS = 111139
    diff = 50/DEGREE_IN_METERS
    
    # Calculate using the formula: length = (population / density) ** 0.5
    city_length = (full_df[city]["population"] / full_df[city]["density"]) ** 0.5
    locations = []
    
    # Search in a square of 110% of the city's theoretical length
    step_length = 0.05 # 50 meters
    grid_length = (city_length * 1.1) / step_length
    for x in range(grid_length):
        for y in range(grid_length):
            locations.append((lat + (x * step_length) - (grid_length * step_length / 2), lng + (y * step_length) - (grid_length * step_length / 2)))

    for location in locations:
        unsigned_url = f"https://maps.googleapis.com/maps/api/streetview/metadata?location={location[0]},{location[1]}&key={GOOGLE_MAPS_API_KEY}"
        signed_url = sign_url(unsigned_url, URL_SIGNING_SECRET)

        response = requests.get(signed_url, stream=True).json()
        # print("response: ", response)
        if response["status"] == "OK":
            return {"city": city_name,
                    "date": response["date"],
                    "lat": response["location"]["lat"],
                    "lng": response["location"]["lng"],
                    "status": "OK"}
        elif response["status"] == "ZERO_RESULTS":
            continue
        else:
            raise Exception("Error: ", response["status"])

    return {"city": city_name,
            "date": None,
            "lat": None,
            "lng": None,
            "status": "NOT_FOUND"}